<a href="https://colab.research.google.com/github/NoCodeProgram/deepLearning/blob/main/cnn/modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt

if torch.backends.mps.is_available():
    my_device = torch.device('mps')
elif torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')

print(my_device)

cuda


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Load CIFAR10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=4)


100%|██████████| 170498071/170498071 [00:13<00:00, 12340887.95it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified


In [3]:

class DoubleConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.block(x)

class ModernGAPCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.features = nn.Sequential(
            DoubleConvBlock(3, 64),
            DoubleConvBlock(64, 128, stride=2),
            DoubleConvBlock(128, 256, stride=2),
        )

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Instantiate the network
net = ModernGAPCNN(num_classes=10)



# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)


In [ ]:
net.to(my_device)
num_epochs = 100
for epoch in range(num_epochs):
    net.train()
    for batch_idx, (data, label) in enumerate(trainloader):
        data, label = data.to(my_device), label.to(my_device)
        scores = net(data)
        loss = criterion(scores, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    net.eval()

    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        for data, label in testloader:
            data, label = data.to(my_device), label.to(my_device)
            scores = net(data)
            loss = criterion(scores, label)
            val_loss += loss.item() * data.size(0)

            predicted = scores.argmax(dim=1)
            correct += predicted.eq(label).sum().item()

        val_loss /= len(testloader.dataset)
        val_accuracy = 100. * correct / len(testloader.dataset)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch [1/100], Training Loss: 0.9581, Validation Loss: 1.0024, Validation Accuracy: 64.01%
Epoch [2/100], Training Loss: 0.7534, Validation Loss: 0.8298, Validation Accuracy: 71.15%
Epoch [3/100], Training Loss: 0.5032, Validation Loss: 0.6752, Validation Accuracy: 76.82%
Epoch [4/100], Training Loss: 0.2928, Validation Loss: 0.6610, Validation Accuracy: 77.19%
Epoch [5/100], Training Loss: 0.9183, Validation Loss: 0.6195, Validation Accuracy: 78.93%
Epoch [6/100], Training Loss: 0.5629, Validation Loss: 0.5699, Validation Accuracy: 80.18%
Epoch [7/100], Training Loss: 0.2366, Validation Loss: 0.5332, Validation Accuracy: 81.81%
Epoch [8/100], Training Loss: 0.5721, Validation Loss: 0.5479, Validation Accuracy: 81.59%
Epoch [9/100], Training Loss: 0.7217, Validation Loss: 0.5546, Validation Accuracy: 81.57%
Epoch [10/100], Training Loss: 0.5403, Validation Loss: 0.5593, Validation Accuracy: 81.57%
Epoch [11/100], Training Loss: 0.5238, Validation Loss: 0.5589, Validation Accuracy: 81.5